5주차 예습과제에서는 6주차 내용인 GPT의 생성 능력을 맛보고자 합니다. 사용할 모델은 koalpaca라는 모델인데, 한국어 버전의 alpaca 모델입니다.

alpaca는 llama라는 모델을 instruction fine-tuning 해서 사람의 지시를 잘 따르는 모델입니다.

llama라는 모델은 LLaMA-13B의 경우 GPT-3보다 10배이상 작음에도 불구하고 대부분의 평가서 GPT-3보다 우수한 성능을 보이며, 더 나아가 LLaMA-65B의 경우 대부분의 벤치마크에서 Chinchilla, Gopher, GPT-3, PaLM와 유사하거나 더 뛰어난 결과를 보이는 훌륭한 모델입니다.

그러면 지금부터 koalpaca로 생성 모델을 한번 경험해보도록 하겠습니다!

In [1]:
# GPU 켜져 있는지 확인

!nvidia-smi

Wed Feb 14 04:33:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are instal

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

peft는 parameter efficient fine-tuning의 약자로, 큰 모델을 코랩에서 돌릴 수 있도록 쪼개서 만든 것입니다.

오래 걸립니다!

In [4]:
from peft import PeftModel, PeftConfig

In [5]:
peft_model_id = "beomi/qlora-koalpaca-polyglot-12.8b-50step"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/52.5k [00:00<?, ?B/s]

model-00001-of-00028.safetensors:   0%|          | 0.00/946M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                

next token generation

In [6]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제를 해결할 수 있다고 말한다. '지능'이라는 단어를 들었을 때 우리는 흔히 IQ를 떠올린다. 하지만 이 책은 IQ가 아닌 '전략적 지능'


 QA

In [7]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [8]:
gen('건강하게 살기 위한 세 가지 방법은?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:447: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 1. 아침에 식사는 반드시 하세요~ㅎㅎㅎ 2. 스트레스를 피하세요~ㅎㅎㅎ 3. 술 담배를 아예 하지마세요~ㅎㅎㅎ 공유하기 글


top_k와 같은 parameter를 조절하면서 실험해보세요!

generation parameters \\
https://huggingface.co/docs/transformers/main_classes/text_generation

1. top_k


*   The number of highest probability vocabulary tokens to keep for top-k-filtering.
*   top_k 샘플링은 텍스트 생성 과정에서 가능한 토큰의 분포에서 상위 k개의 가장 가능성이 높은 토큰만을 고려하는 방법. top_k 파라미터는 k의 값을 결정.
*   default: 50
*   값이 높아질수록 더 다양하고 창의적인 텍스트를 생성하나, 너무 높게 설정 시 문맥에 맞지 않거나 일관성이 없는 텍스트를 생성하는 문제 발생
*   값이 낮아질수록 일관성 있는 텍스트를 생성하나, 너무 낮게 설정 시 단순하거나 뻔한 텍스트를 생성하는 문제 발생.



In [43]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, max_length=64, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결하는 '인공지능'의 연구개발이 가속화되면서 현재 알파고 이후 인공지능에 대한 사람들의 관심이 뜨거워지고 있다.이 책은 인공지능의 개념과 역사


In [44]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, max_length=64, top_k=100)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제를 해결하려는 '구글' '애플'등.....이들 브랜드의 이름은 '브랜드' 단어를 연상시키기보다 '인간'이라는 이름이 더 연상되는 것입니다.


In [45]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, max_length=64, top_k=20)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 여러가지 수수께끼를 풀어줄 것"이라며 기대감을 표시했습니다. ​​또한, 그는 "로봇의 발달로 인해 인간이 하기 싫어하는 일, 어려운 일, 위험부담


2. temperature
*   The value used to modulate the next token probabilities.
*   텍스트를 생성할 때 사용되는 확률 분포를 조절하여 텍스트의 다양성과 예측 가능성을 조정하는 파라미터
*   default = 1
*   낮을 수록 확실성 높은 텍스트 생성
*   높을 수록 다양한 텍스트 생성










In [30]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.5, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제를 해결할 수 있는 실마리를 얻을 수 있을 것이다.​​​​​​​​​​​​�


In [26]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=1, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결하기 위해 많은 도전을 하고 있습니다.인공지능 분야에는 자율주행차,로봇산업,드론,IoT/Iot,인공지능 의료기기 등 이 있습니다.현재 많


In [32]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=1.5, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 삶의 비밀을 밝힐 유일한 실마리"라고 설명하고 있답니다! ▲  ​영화 예고편 입니다 영화정보​영화 은 ​2월 7일 목요일에 공개될 ​기대작중
